### Problem 2: Data management
In the data_management folder you will find an [Excel file](data_management/data_2019.xls) containing data 
for 1 year of electricity consumption collected by billing information.

We need to transform this data, by creating a time-series with regular daily frequency. If gaps are detected,
they must appear as NaN values. You can use any
python library like pandas to transform this data.

In [1]:
import pandas as pd
import time
import duckdb 

In [2]:
df = pd.read_excel("data_2019.xls",usecols=["d_ini","d_end","kwh"]) # read excel
df = df.dropna() # delete missing values

In [3]:
def update_values(row):
    dates.loc[row['d_ini']:row['d_end']] = row['kwh']/ ((row['d_end'] - row['d_ini']).days + 1)

start = time.time()
dates = pd.Series(index=pd.date_range(start=df['d_ini'].min(), end=df['d_end'].max(), freq='D'), name="kwh") # create the daily frequency
df.apply(update_values,axis = 1)
    
end = time.time()
1000*(end-start)

5.877971649169922

DuckDB store the daily data `dates` in a persistent table called consumption at `../rest_api/electricity.db` (problem 3)

In [4]:
def df_to_DBtable(df, table_name, NEW_DB=":default:"):
    con = duckdb.connect(NEW_DB)
    con.execute(f'CREATE OR REPLACE TABLE {table_name} AS SELECT * FROM {df}')
    con.close()
    
df_dates = pd.DataFrame({'date':dates.index,'kwh':dates.values})
df_to_DBtable("df_dates","consumption","../rest_api/electricity.db")